In [1]:
!pip install tensorflow-recommenders
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [2]:
# import required packages
# utility
import sys
import re
import html
import pprint
import numpy as np
import pandas as pd
from typing import Dict, Text
from sklearn.preprocessing import StandardScaler

# tensorflow
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [3]:
# # configure gpu
# gpus = tf.config.list_physical_devices("GPU")

# print("Num GPUs Available: ", len(gpus))

# # if gpus:
# #     # Create 2 virtual GPUs with 4GB memory each
# #     try:
# #         tf.config.set_logical_device_configuration(gpus[0],[tf.config.LogicalDeviceConfiguration(memory_limit=4096), tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
# #         logical_gpus = tf.config.list_logical_devices("GPU")
# #         print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
# #     except RuntimeError as e:
# #         # Virtual devices must be set before GPUs have been initialized
# #         print(e)

# strategy = tf.distribute.MirroredStrategy()

In [4]:
# # configure tpu
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(resolver)

# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))

# strategy = tf.distribute.TPUStrategy(resolver)

## EDA and Preprocessing: Anime Recommendations Dataset
- Ratings data
- Metadata

### Ratings data

In [5]:
# load ratings dataset
# kaggle path
# rating_df = pd.read_csv('/kaggle/input/anime-recommendations-database/rating.csv')

# local path
rating_df = pd.read_csv('../data/rating.csv')

rating_df.shape

(7813737, 3)

In [6]:
# first 5 data points
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [7]:
# info
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


In [8]:
# convert id columns to string
rating_df['user_id'] = rating_df['user_id'].map(lambda x: str(x))
rating_df['anime_id'] = rating_df['anime_id'].map(lambda x: str(x))

In [9]:
rating_df.isna().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

### Metadata

In [10]:
# load anime metadata
# kaggle path
# anime_df = pd.read_csv('/kaggle/input/anime-recommendations-database/anime.csv')

# local path
rating_df = pd.read_csv('../data/anime.csv')

anime_df.shape

(12294, 7)

In [11]:
# first 5 data points
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [12]:
# drop useless columns
# members
anime_df = anime_df.drop(['members'], axis=1)

In [13]:
# check null values
anime_df.isna().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
dtype: int64

In [14]:
# replace null values for genre with mode
anime_df['genre'] = anime_df['genre'].fillna(anime_df['genre'].mode()[0])

# replace null values for type with mode
anime_df['type'] = anime_df['type'].fillna(anime_df['type'].mode()[0])

# replace null value for rating with mean
anime_df['rating'] = anime_df['rating'].fillna(anime_df['rating'].mean())

In [15]:
# verify null values handling
anime_df.isna().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
dtype: int64

In [16]:
# info
anime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12294 non-null  object 
 3   type      12294 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12294 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 576.4+ KB


In [17]:
# convert anime_id column to string
anime_df['anime_id'] = anime_df['anime_id'].apply(lambda x: str(x))

In [18]:
# sample anime name with invalid characters
anime_df['name'].loc[8441]

'Dennou Sentai Voogie&#039;s★Angel: Forever and Ever'

In [19]:
# clean anime name column
# remove html encodings
# convert to regular characters
anime_df['name'] = anime_df['name'].apply(html.unescape)

# remove emojis and other miscellaneous unicode characters
# encode as ascii to remove miscellaneous unicode characters
# replaces the characters with '?'
# converts str to array of bytes
anime_df['name'] = anime_df['name'].map(lambda x: x.encode('ascii', 'replace'))

# decode to convert to str
anime_df['name'] = anime_df['name'].map(lambda x: x.decode())

# replace '?' with a space
anime_df['name'] = anime_df['name'].map(lambda x: x.replace('?', ' '))

# convert to lower case
anime_df['name'] = anime_df['name'].map(lambda x: x.lower())

# remove punctuations
anime_df['name'] = anime_df['name'].map(lambda x: re.sub(r'[^\w\s]', ' ', x))

# remove single length words
anime_df['name'] = anime_df['name'].map(lambda x: re.sub(r'\b\w{1}\b', '', x))

# remove extra spaces
anime_df['name'] = anime_df['name'].map(lambda x: re.sub(r' +', ' ', x))

In [20]:
# verify cleaning of anime name column
anime_df['name'].loc[8441]

'dennou sentai voogie angel forever and ever'

In [21]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating
0,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37
1,5114,fullmetal alchemist brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26
2,28977,gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25
3,9253,steins gate,"Sci-Fi, Thriller",TV,24,9.17
4,9969,gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16


In [22]:
# clean and normalize genre column
# convert to lower case
anime_df['genre'] = anime_df['genre'].map(lambda x: x.lower())

# remove punctuations
# keep hyphen(-) for sci-fi
anime_df['genre'] = anime_df['genre'].map(lambda x: re.sub(r'[^\w\s-]', ' ', x))

# remove single length characters
anime_df['genre'] = anime_df['genre'].map(lambda x: re.sub(r'\b\w{1}\b', '', x))

# remove extra spaces
anime_df['genre'] = anime_df['genre'].map(lambda x: re.sub(r' +', ' ', x))

In [23]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating
0,32281,kimi no na wa,drama romance school supernatural,Movie,1,9.37
1,5114,fullmetal alchemist brotherhood,action adventure drama fantasy magic military ...,TV,64,9.26
2,28977,gintama,action comedy historical parody samurai sci-fi...,TV,51,9.25
3,9253,steins gate,sci-fi thriller,TV,24,9.17
4,9969,gintama,action comedy historical parody samurai sci-fi...,TV,51,9.16


In [24]:
# normalize type column
# convert to lower case
anime_df['type'] = anime_df['type'].map(lambda x: x.lower())

# no further cleaning required for type column

In [25]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating
0,32281,kimi no na wa,drama romance school supernatural,movie,1,9.37
1,5114,fullmetal alchemist brotherhood,action adventure drama fantasy magic military ...,tv,64,9.26
2,28977,gintama,action comedy historical parody samurai sci-fi...,tv,51,9.25
3,9253,steins gate,sci-fi thriller,tv,24,9.17
4,9969,gintama,action comedy historical parody samurai sci-fi...,tv,51,9.16


In [26]:
# anime_df['episodes'] = anime_df['episodes'].apply(lambda x: float(x))
# errorneous value for episodes column
anime_df[anime_df['episodes'] == 'Unknown']

,anime_id,name,genre,type,episodes,rating
74,21,one piece,action adventure comedy drama fantasy shounen ...,tv,Unknown,8.580000
252,235,detective conan,adventure comedy mystery police shounen,tv,Unknown,8.250000
615,1735,naruto shippuuden,action comedy martial arts shounen super power,tv,Unknown,7.940000
991,966,crayon shin chan,comedy ecchi kids school shounen slice of life,tv,Unknown,7.730000
1021,33157,tanaka kun wa itsumo kedaruge specials,comedy school slice of life,special,Unknown,7.720000
...,...,...,...,...,...,...
12265,34361,kyonyuu reijou mc gakuen,hentai,ova,Unknown,6.473902
12274,34492,nuki doki tenshi to akuma no sakusei battle re...,hentai,ova,Unknown,6.473902
12280,34312,saimin class,hentai,ova,Unknown,6.473902
12282,34388,shikkoku no shaga the animation,hentai,ova,Unknown,6.473902


In [27]:
# handle errorneous .aggregatealue for episodes column
anime_df['episodes'] = anime_df['episodes'].replace('Unknown', str(sys.maxsize))

# convert episodes column from string to float
anime_df['episodes'] = anime_df['episodes'].apply(lambda x: float(x))

In [28]:
# rename rating column to avoid confusion with user rating column
anime_df = anime_df.rename({'rating': 'score'}, axis=1)

In [29]:
anime_df.head()

,anime_id,name,genre,type,episodes,score
0,32281,kimi no na wa,drama romance school supernatural,movie,1.0,9.37
1,5114,fullmetal alchemist brotherhood,action adventure drama fantasy magic military ...,tv,64.0,9.26
2,28977,gintama,action comedy historical parody samurai sci-fi...,tv,51.0,9.25
3,9253,steins gate,sci-fi thriller,tv,24.0,9.17
4,9969,gintama,action comedy historical parody samurai sci-fi...,tv,51.0,9.16


In [30]:
# add columns from metadata to ratings dataframe
rating_df = pd.merge(rating_df, anime_df, on='anime_id')

rating_df.shape

(7813727, 8)

In [31]:
# first 5 data points
rating_df.head()

,user_id,anime_id,rating,name,genre,type,episodes,score
0,1,20,-1,naruto,action comedy martial arts shounen super power,tv,220.0,7.81
1,3,20,8,naruto,action comedy martial arts shounen super power,tv,220.0,7.81
2,5,20,6,naruto,action comedy martial arts shounen super power,tv,220.0,7.81
3,6,20,-1,naruto,action comedy martial arts shounen super power,tv,220.0,7.81
4,10,20,-1,naruto,action comedy martial arts shounen super power,tv,220.0,7.81


In [32]:
# convert rating data from dataframe to tensorflow Dataset
ratings = tf.data.Dataset.from_tensor_slices(dict(rating_df))

# keep only user_id and anime_name column
ratings = ratings.map(lambda x: {
    'user_id': x['user_id'],
    'anime_title': x['name'],
    'genre': x['genre'],
    'type': x['type'],
    'episodes': x['episodes'],
    'score': x['score'],
    'user_rating': x['rating']
})

for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'anime_title': b'naruto',
 'episodes': 220.0,
 'genre': b'action comedy martial arts shounen super power',
 'score': 7.81,
 'type': b'tv',
 'user_id': b'1',
 'user_rating': -1}


In [33]:
# convert anime metadata from dataframe to tensorflow Dataset
animes = tf.data.Dataset.from_tensor_slices(dict(anime_df[['name', 'genre', 'type', 'episodes', 'score']].rename({'name': 'anime_title'}, axis=1)))
animes = animes.map(lambda x: x['anime_title'])

for x in animes.take(1).as_numpy_iterator():
    pprint.pprint(x)

b'kimi no na wa '


In [34]:
# create vocabulary of all features
# vocabulary of user ids
vocab_user_ids = rating_df['user_id'].unique().astype(str)

# vocabulary of anime titles
vocab_anime_titles = rating_df['name'].unique().astype(str)

# vocabulary of genres
vocab_genres = rating_df['genre'].unique().astype(str)

# vocabulary of type
vocab_types = rating_df['type'].unique().astype(str)

# vocabulary of episodes
vocab_episodes = rating_df['episodes'].unique().astype(float)

# vocabulary of score
vocab_scores = rating_df['score'].unique().astype(float)

In [35]:
# shuffle and split data: train, valid, test
# set seed
tf.random.set_seed(42)

# total data points
N = rating_df.shape[0]

# total train data points
N_train = int(0.6 * N)

# total valid data points
N_valid = int(0.2 * N)

# total test data points
N_test = N - (N_train + N_valid)

# shuffle data
shuffled = ratings.shuffle(N, seed=42, reshuffle_each_iteration=False)

# # split data
train = shuffled.take(N_train)
valid = shuffled.skip(N_train).take(N_valid)
test = shuffled.skip((N_train + N_valid)).take(N_test)

## Ranking Model using Metadata

### Model Architecture: Base Ranking Model

In [36]:
# Base Ranking Model
class RankingModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        # user_id embedding
        self.user_id_embedding = tf.keras.Sequential([
          tf.keras.layers.StringLookup(vocabulary=vocab_user_ids, mask_token=None),
          tf.keras.layers.Embedding(len(vocab_user_ids) + 1, embedding_dimension)
        ])
        
        # anime title embedding
        self.anime_title_embedding = tf.keras.Sequential([
          tf.keras.layers.StringLookup(vocabulary=vocab_anime_titles, mask_token=None),
          tf.keras.layers.Embedding(len(vocab_anime_titles) + 1, embedding_dimension)
        ])
        
        # meta embeddings
        # genre embedding
        self.genre_embedding = tf.keras.Sequential([
          tf.keras.layers.StringLookup(vocabulary=vocab_genres, mask_token=None),
          tf.keras.layers.Embedding(len(vocab_genres) + 1, embedding_dimension)
        ])
        
        # type embedding
        self.type_embedding = tf.keras.Sequential([
          tf.keras.layers.StringLookup(vocabulary=vocab_types, mask_token=None),
          tf.keras.layers.Embedding(len(vocab_types) + 1, embedding_dimension)
        ])
        
        # episodes embedding
        self.episodes_embedding = tf.keras.Sequential([
            tf.keras.layers.IntegerLookup(vocabulary=vocab_episodes, mask_value=None),
            tf.keras.layers.Embedding(len(vocab_episodes) + 1, embedding_dimension)
        ])
        
        # score embedding
        self.score_embedding = tf.keras.Sequential([
            tf.keras.layers.Embedding(len(vocab_scores) + 1, embedding_dimension)
        ])

        # predicts ratings
        self.ratings = tf.keras.Sequential([
          tf.keras.layers.Dense(256, activation="relu"),
          tf.keras.layers.Dense(128, activation="relu"),
          tf.keras.layers.Dense(64, activation="relu"),
          tf.keras.layers.Dense(1)
        ])

    def call(self, inputs):
        user_ids, anime_titles, genres, types, episodes, scores  = inputs

        # generate embedding for the user ids
        user_id_embedding = self.user_id_embedding(user_ids)
        
        # generate embedding for the anime titles
        anime_title_embedding = self.anime_title_embedding(anime_titles)
        
        # generate embedding for the genres
        genre_embedding = self.genre_embedding(genres)
        
        # generate embedding for the types
        type_embedding = self.type_embedding(types)
        
        # generate embedding for episodes
        episodes_embedding = self.episodes_embedding(episodes)
        
        # generate embedding for scores
        score_embedding = self.score_embedding(scores)

        # predict and return the ratings
        return self.ratings(tf.concat([
            user_id_embedding, 
            anime_title_embedding, 
            genre_embedding, 
            type_embedding,
            episodes_embedding,
            score_embedding
        ], axis=1))

### Model Architecture: AnimeRankingModel

In [37]:
# wrapper for Base Ranking Model
class AnimeRankingModel(tfrs.models.Model):
    def __init__(self):
        super().__init__()
        
        self.ranking_model: tf.keras.Model = RankingModel()
            
        self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
          loss = tf.keras.losses.MeanSquaredError(),
          metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features: Dict[str, tf.Tensor]):
        return self.ranking_model((
            features['user_id'], 
            features['anime_title'], 
            features['genre'], 
            features['type'],
            features['episodes'],
            features['score']
        ))

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False):
        labels = features.pop('user_rating')

        rating_predictions = self(features)

        # computes the loss and the metrics
        return self.task(labels=labels, predictions=rating_predictions)

In [38]:
model = AnimeRankingModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

In [39]:
# prepare train and validation sets
cached_train = train.shuffle(N_train).batch(8192).cache()
cached_valid = valid.batch(4096).cache()

In [40]:
history = model.fit(cached_train, epochs=5)

Epoch 1/5
573/573 [==============================] - 296s 111ms/step - root_mean_squared_error: 2.3596 - loss: 5.5648 - regularization_loss: 0.0000e+00 - total_loss: 5.5648
Epoch 2/5
573/573 [==============================] - 48s 84ms/step - root_mean_squared_error: 2.1097 - loss: 4.4490 - regularization_loss: 0.0000e+00 - total_loss: 4.4490
Epoch 3/5
573/573 [==============================] - 48s 83ms/step - root_mean_squared_error: 2.0471 - loss: 4.1879 - regularization_loss: 0.0000e+00 - total_loss: 4.1879
Epoch 4/5
573/573 [==============================] - 48s 84ms/step - root_mean_squared_error: 2.0044 - loss: 4.0150 - regularization_loss: 0.0000e+00 - total_loss: 4.0150
Epoch 5/5
573/573 [==============================] - 48s 83ms/step - root_mean_squared_error: 1.9503 - loss: 3.7992 - regularization_loss: 0.0000e+00 - total_loss: 3.7992


In [41]:
# evaluate the model
model.evaluate(cached_valid, return_dict=True)

382/382 [==============================] - 240s 54ms/step - root_mean_squared_error: 2.1857 - loss: 4.7770 - regularization_loss: 0.0000e+00 - total_loss: 4.7770


{'root_mean_squared_error': 2.185739517211914,
 'loss': 4.657506942749023,
 'regularization_loss': 0,
 'total_loss': 4.657506942749023}

In [42]:
# test_ratings = {}
# test_anime_titles = ['Oppai Infinity ! The Animation']
# for anime_title in test_anime_titles:
#     test_ratings[anime_title] = model({
#         "user_id": np.array(["21060"]),
#         "anime_title": np.array([anime_title]),
#         "genre":np.array(['comedy romance']),
#         'type': np.array(['tv'])
#     })

# print("Ratings:")
# for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
#   print(f"{title}: {score}")